In [2]:
# chat_agent.py

import os
from dotenv import load_dotenv
load_dotenv()
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")


import time
import yfinance as yf
import wikipedia
from langchain_google_genai import ChatGoogleGenerativeAI

class Chatbot:
    def __init__(self, system_prompt: str = ""):
        self.messages = []
        if system_prompt:
            self.messages.append({"role": "system", "content": system_prompt})

    def __call__(self, user_input: str) -> str:
        # 1) Append the user message
        self.messages.append({"role": "user", "content": user_input})
        # 2) Invoke the LLM
        assistant_response = self._invoke_llm()
        # 3) Append the assistant’s raw response
        self.messages.append({"role": "assistant", "content": assistant_response})
        # 4) If the LLM inserted "PAUSE", cut off there
        if "PAUSE" in assistant_response:
            return assistant_response.split("PAUSE", 1)[0].strip()
        return assistant_response

    def _invoke_llm(self) -> str:
        llm = ChatGoogleGenerativeAI(
            model="gemini-2.0-flash",
            google_api_key=os.getenv("GOOGLE_API_KEY")
        )
        print("→ Invoking LLM with messages:", self.messages)
        start = time.time()
        result = llm.invoke(self.messages)
        elapsed = time.time() - start
        print(f"← LLM returned in {elapsed:.1f}s")
        return result.content

# --- Tool implementations ---

def simon_blog_search(query: str) -> str:
    # TODO: replace with real scraping logic
    return f"[Simon’s blog search result for '{query}']"

def wiki_search(query: str) -> str:
    try:
        return wikipedia.summary(query, sentences=2)
    except Exception as e:
        return f"[Wikipedia error: {e}]"

def get_indian_stock_price(ticker: str) -> str:
    # e.g. ticker="RELIANCE.NS"
    try:
        df = yf.Ticker(ticker).history(period="1d")
        price = df["Close"].iloc[-1]
        return f"{ticker} closed at ₹{price:.2f}"
    except Exception as e:
        return f"[Yahoo Finance error: {e}]"

# --- Simple agent loop ---

def run_agent_query(bot: Chatbot, question: str):
    print("User:", question)
    response = bot(question)
    print("Assistant:", response)

    # If the assistant decided on a tool:
    if "Action:" in response:
        action_line = response.split("Action:")[1].splitlines()[0].strip()
        name, _, arg = action_line.partition(":")
        arg = arg.strip()
        if name == "simon_blog_search":
            tool_out = simon_blog_search(arg)
        elif name == "wiki_search":
            tool_out = wiki_search(arg)
        elif name == "yahoo_finance":
            tool_out = get_indian_stock_price(arg)
        else:
            tool_out = f"[Unknown tool: {name}]"
        print("Tool result:", tool_out)

        # Feed the tool result back to the bot and get final answer
        bot.messages.append({"role": "tool", "content": tool_out})
        final = bot("")  # empty user_input means “continue”
        print("Final Answer:", final)

if __name__ == "__main__":
    # Example usage
    system_prompt = (
        "You can call tools by writing:\n"
        "Action: simon_blog_search: <query>\n"
        "Action: wiki_search: <query>\n"
        "Action: yahoo_finance: <NSE ticker>\n"
        "When you need to pause and wait for the tool, include 'PAUSE' in your response.\n"
    )
    bot = Chatbot(system_prompt)
    bot("What is the current price of Asian Paints?")


→ Invoking LLM with messages: [{'role': 'system', 'content': "You can call tools by writing:\nAction: simon_blog_search: <query>\nAction: wiki_search: <query>\nAction: yahoo_finance: <NSE ticker>\nWhen you need to pause and wait for the tool, include 'PAUSE' in your response.\n"}, {'role': 'user', 'content': 'What is the current price of Asian Paints?'}]
← LLM returned in 2.2s
